# tiling inference

回歸正常的流程，就是拿到大圖才tiling，tiling inference each image 再combine


In [ ]:
%matplotlib inline

# import importlib
# importlib.reload(util)

# read in tf train example
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from util import cur_path, to_abs_path, get_jpg_paths_from_dir, to_aidea_save, zip_dir, get_edge_tile_px, split_img
import util 
import zipfile
import cv2
import io
import tensorflow as tf
from object_detection.utils import label_map_util, np_box_list, np_box_list_ops, dataset_util
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

In [ ]:
IMAGE_DIR_LIST=[util.to_abs_path( pathlib.PurePath( 'data/test_public'), util.cur_path.parent),
               util.to_abs_path( pathlib.PurePath( 'data/test_private'), util.cur_path.parent),
               ]

IMAGE_PATHS_LIST=[ util.get_jpg_paths_from_dir(i) for i in IMAGE_DIR_LIST ]

In [ ]:
#PATH_TO_MODEL_DIR = '/mnt/c/Users/insleker/Google Drive/workspace/riceCV/code/exported_models/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8'
PATH_TO_MODEL_DIR = '/mnt/c/Users/insleker/Google Drive/workspace/riceCV/code/exported_models/tile_centernet_resnet50_v1_fpn_512x512_coco17_tpu-8'
#PATH_TO_MODEL_DIR = '/mnt/c/Users/insleker/Google Drive/workspace/riceCV/code/exported_models/centernet_hg104_512x512_coco17_tpu-8'

model_p= pathlib.PurePath(PATH_TO_MODEL_DIR)
MODEL_NAME= model_p.parts[-1]

LABEL_FILENAME = 'rice_label_map.pbtxt'
PATH_TO_LABELS = '/mnt/c/Users/insleker/Google Drive/workspace/riceCV/data/rice_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [ ]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
'''
modify ver, which iou become only area of smaller box as denominator
after calculate the output num, it seem have bug
'''
def nms(rects, thd=0.5):
    """
    Filter rectangles
    rects is array of oblects ([x1,y1,x2,y2], confidence, class)
    thd - intersection threshold (intersection divides min square of rectange)
    """
    out = []

    remove = [False] * len(rects)

    for i in range(0, len(rects) - 1):
        if remove[i]:
            continue
        inter = [0.0] * len(rects)
        for j in range(i, len(rects)):
            if remove[j]:
                continue
            inter[j] = intersection(rects[i][0], rects[j][0]) / min(square(rects[i][0]), square(rects[j][0]))

        max_prob = 0.0
        max_idx = 0
        for k in range(i, len(rects)):
            if inter[k] >= thd:
                if rects[k][1] > max_prob:
                    max_prob = rects[k][1]
                    max_idx = k

        for k in range(i, len(rects)):
            if (inter[k] >= thd) & (k != max_idx):
                remove[k] = True

    for k in range(0, len(rects)):
        if not remove[k]:
            out.append(rects[k])

    boxes = [box[0] for box in out]
    scores = [score[1] for score in out]
    classes = [cls[2] for cls in out]
    return boxes, scores, classes


def intersection(rect1, rect2):
    """
    Calculates square of intersection of two rectangles
    rect: list with coords of top-right and left-boom corners [x1,y1,x2,y2]
    return: square of intersection
    """
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]));
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]));
    overlapArea = x_overlap * y_overlap;
    return overlapArea


def square(rect):
    """
    Calculates square of rectangle
    """
    return abs(rect[2] - rect[0]) * abs(rect[3] - rect[1])

In [ ]:
a=np.array([1,3])
np.max(a)

In [ ]:
from object_detection.utils.np_box_ops import area, intersection, iou, ioa
from object_detection.utils.np_box_list_ops import filter_scores_greater_than, sort_by_field, gather
# base on original non max suppression, but change the iou to ioa with larger value

def modify_non_max_suppression(boxlist,
                               max_output_size=10000,
                               ioa_threshold=1.0,
                               score_threshold=-10.0):
  """Non maximum suppression.

  This op greedily selects a subset of detection bounding boxes, pruning
  away boxes that have high IOU (intersection over union) overlap (> thresh)
  with already selected boxes. In each iteration, the detected bounding box with
  highest score in the available pool is selected.

  Args:
    boxlist: BoxList holding N boxes.  Must contain a 'scores' field
      representing detection scores. All scores belong to the same class.
    max_output_size: maximum number of retained boxes
    iou_threshold: intersection over union threshold.
    score_threshold: minimum score threshold. Remove the boxes with scores
                     less than this value. Default value is set to -10. A very
                     low threshold to pass pretty much all the boxes, unless
                     the user sets a different score threshold.

  Returns:
    a BoxList holding M boxes where M <= max_output_size
  Raises:
    ValueError: if 'scores' field does not exist
    ValueError: if threshold is not in [0, 1]
    ValueError: if max_output_size < 0
  """
  if not boxlist.has_field('scores'):
    raise ValueError('Field scores does not exist')
  if ioa_threshold < 0. or ioa_threshold > 1.0:
    raise ValueError('IOA threshold must be in [0, 1]')
  if max_output_size < 0:
    raise ValueError('max_output_size must be bigger than 0.')

  boxlist = filter_scores_greater_than(boxlist, score_threshold)
  if boxlist.num_boxes() == 0:
    return boxlist

  boxlist = sort_by_field(boxlist, 'scores')

  # Prevent further computation if NMS is disabled.
  if ioa_threshold == 1.0:
    if boxlist.num_boxes() > max_output_size:
      selected_indices = np.arange(max_output_size)
      return gather(boxlist, selected_indices)
    else:
      return boxlist

  boxes = boxlist.get()
  num_boxes = boxlist.num_boxes()
  # is_index_valid is True only for all remaining valid boxes,
  is_index_valid = np.full(num_boxes, 1, dtype=bool)
  selected_indices = []
  num_output = 0
  for i in range(num_boxes):
    if num_output < max_output_size:
      if is_index_valid[i]:
        num_output += 1
        selected_indices.append(i)
        is_index_valid[i] = False
        valid_indices = np.where(is_index_valid)[0]
        if valid_indices.size == 0:
          break

        intersect_over_union =  ioa(np.expand_dims(boxes[i, :], axis=0), boxes[valid_indices, :])
        intersect_over_union = np.squeeze(intersect_over_union, axis=0)
        is_index_valid[valid_indices] = np.logical_and(
            is_index_valid[valid_indices],
            intersect_over_union <= ioa_threshold)
        
  return gather(boxlist, np.array(selected_indices))

In [ ]:
%matplotlib inline

In [ ]:
TRESH= 0.25 #0.35
T_ROW=5
T_COL=8
OVLP_PCT=1/4

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for IMAGE_PATHS in IMAGE_PATHS_LIST:
    for image_path in IMAGE_PATHS[:]: #can control image number

        print('Running inference for {}... '.format(image_path))

        image_np = load_image_into_numpy_array(image_path)
        
        #do tiling
        boxlist=np_box_list.BoxList( np.array([], dtype=np.float).reshape(0,4) ) #[y_min, x_min, y_max, x_max]
        boxlist.add_field('scores', np.array([], dtype=np.float) )
        sub_img_list, _= split_img(image_np,None,T_ROW,T_COL,OVLP_PCT)
        for i in range(T_ROW):
            for j in range(T_COL):
                sub_img=sub_img_list[i][j]
                
                # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
                input_tensor = tf.convert_to_tensor(sub_img)
                # The model expects a batch of images, so add an axis with `tf.newaxis`.
                input_tensor = input_tensor[tf.newaxis, ...]

                # input_tensor = np.expand_dims(image_np, 0)
                detections = detect_fn(input_tensor)

                # All outputs are batches tensors.
                # Convert to numpy arrays, and take index [0] to remove the batch dimension.
                # We're only interested in the first num_detections.
                num_detections = int(detections.pop('num_detections'))
                detections = {key: value[0, :num_detections].numpy()
                               for key, value in detections.items()}
                detections['num_detections'] = num_detections

                #viz to check predict
                #detection_classes should be ints.
#                 detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
#                 image_np_with_detections = sub_img.copy()
#                 viz_utils.visualize_boxes_and_labels_on_image_array(
#                       image_np_with_detections,
#                       detections['detection_boxes'],
#                       detections['detection_classes'],
#                       detections['detection_scores'],
#                       category_index,
#                       use_normalized_coordinates=True,
#                       max_boxes_to_draw=300,
#                       min_score_thresh=TRESH)
#                 plt.figure( figsize=(6, 6) )
#                 plt.imshow(image_np_with_detections)
#                 plt.show()
#                 print('Done')
                
                #reverse boxes to ori coordinate
                h,w,c= image_np.shape
                hh,ww,cc= sub_img.shape
                WITH_OVLP_N_ROW, N_ROW= get_edge_tile_px(S=h, T=T_ROW, OVLP_PCT=1/4)
                WITH_OVLP_N_COL, N_COL= get_edge_tile_px(S=w, T=T_COL, OVLP_PCT=1/4)
                b_y_min= WITH_OVLP_N_ROW*i
                b_y_max= b_y_min+N_ROW
                b_x_min= WITH_OVLP_N_COL*j
                b_x_max= b_x_min+N_COL
                window= [-b_y_min/hh, -b_x_min/ww, (h-b_y_min)/hh, (w-b_x_min)/ww] #normalized
                
                framed_boxes= np_box_list.BoxList(detections['detection_boxes'])
                framed_boxes= np_box_list_ops.change_coordinate_frame( framed_boxes, window )
                boxlist.data['boxes']= np.concatenate( (boxlist.get(), framed_boxes.get()) )
                boxlist.data['scores']= np.concatenate( (boxlist.get_field('scores'), detections['detection_scores']) )
                
        #non max supression all the boxes in boxlist
        print('boxes num before nms:', boxlist.num_boxes() )
        
#         boxlist= np_box_list_ops.non_max_suppression(boxlist, 
#                                                      max_output_size=4500,
#                                                      iou_threshold=0.2, 
#                                                      score_threshold=TRESH)
        boxlist= modify_non_max_suppression(boxlist, 
                                             max_output_size=4500,
                                             ioa_threshold=0.4, 
                                             score_threshold=TRESH)

#         out_box_list=[]
#         for i in range(boxlist.num_boxes()):
#             window=boxlist.get()[i]
#             out_box_list.append( [[window[1],window[0],window[3],window[2]], boxlist.get_field('scores')[i], 1] )
#         boxes, scores, classes= nms(out_box_list, thd=0.3)
#         print()
#         boxes=np.array(boxes)
#         print(boxes.shape)
#         boxes= np.stack( [boxes[:,1],boxes[:,0],boxes[:,3],boxes[:,2] ], axis=1 )
#         print(boxes.shape)
#         boxlist=np_box_list.BoxList( boxes ) #[y_min, x_min, y_max, x_max]
#         boxlist.add_field('scores', np.array(scores) )
        
        print('boxes num aft nms:', boxlist.num_boxes() )

        
            
        #--- save to aidea data, which use score and tresh to filter boxes
        print('detect size before thresh:', detections['detection_boxes'].shape)
        img_p= pathlib.PurePath(image_path)
        img_name= img_p.stem
        target_path= img_p.parents[0].joinpath('inference/tli_'+MODEL_NAME+'/'+img_name+'.csv')
        print(target_path)
        pathlib.Path(target_path.parent).mkdir(parents=True, exist_ok=True) #create folder if not exists
        detection_boxes, detection_centers= to_aidea_save(image_np, boxlist.get(), \
                                                          boxlist.get_field('scores'), target_path, thresh=TRESH)

        #--- visualize the result
#         image_np_with_detections = image_np.copy()
#         viz_utils.visualize_boxes_and_labels_on_image_array(
#                       image_np_with_detections,
#                       boxlist.get(),
#                       [1]*boxlist.num_boxes(),
#                       boxlist.get_field('scores'),
#                       category_index,
#                       use_normalized_coordinates=True,
#                       max_boxes_to_draw=4500,
#                       min_score_thresh=TRESH)

#         plt.figure( figsize=[6.4*4, 4.8*4] )
#         plt.imshow(image_np_with_detections)
#         plt.show()
        print()
    

print('finish')

In [ ]:
zip_path= cur_path.joinpath( 'AIdea_inference_tl_{}.zip'.format(MODEL_NAME) )
with zipfile.ZipFile( zip_path, mode='w' ) as zf:
    zf.close()
for IMAGE_DIR in IMAGE_DIR_LIST:
    csvs_dir=IMAGE_DIR.joinpath('inference/tli_'+MODEL_NAME)
    print(csvs_dir)
    zip_dir( zip_path, csvs_dir, '**/*.csv')
    
print('finsih zip file, upload plz')

In [ ]:
IMAGE_DIR_LIST